In [8]:
%pip install requests beautifulsoup4 pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
ticker = 'MSFT'

In [11]:
def fetch_finviz_news(stock_symbol):
    url = f"https://finviz.com/quote.ashx?t={stock_symbol}"
    headers = {'User-Agent': 'Mozilla/5.0'}  # Some websites require a user-agent header
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        headlines = []

        # Attempt to locate the news table by its class. Update if the class name has changed.
        news_table = soup.find('table', class_='fullview-news-outer')
        if news_table:
            for row in news_table.findAll('tr'):
                try:
                    text = row.a.text.strip() if row.a else 'No headline available'
                    link = row.a['href'] if row.a else 'No link available'
                    date = row.td.text.strip().split('\n')[0] if row.td else 'No date available'
                    headlines.append({'headline': text, 'link': link, 'date': date})
                except AttributeError:
                    continue  # Skip rows where expected elements are missing
        else:
            return "News table not found."

        return headlines
    else:
        return "Failed to retrieve news."

In [12]:
# Fetch news for AAPL
news = fetch_finviz_news(ticker)

# Convert to DataFrame for easier viewing
news_df = pd.DataFrame(news)

# Display the first few rows
print(news_df.head())

                                            headline  \
0  Magnificent Seven Stocks To Buy And Watch: Nvi...   
1   Jim Cramer Says You Should Avoid These 11 Stocks   
2  These Stocks Are Moving the Most Today: Boeing...   
3  Nvidia Has a Secret Weapon to Keep Growing Lon...   
4  Intel, AMD Fall on Report China to Limit Use o...   

                                                link           date  
0  https://finance.yahoo.com/m/4205eaa9-f620-3a0b...  Today 12:23PM  
1  https://finance.yahoo.com/news/jim-cramer-says...        11:45AM  
2  https://finance.yahoo.com/m/ef62b59f-2890-3f7a...        11:17AM  
3  https://finance.yahoo.com/m/cc99c582-04fa-3d85...        10:31AM  
4  https://finance.yahoo.com/news/intel-amd-fall-...        10:16AM  


In [13]:
news_df.to_csv(f'{ticker}_stock_news.csv', index=False)
print(f"{ticker} stock news saved to aapl_stock_news.csv.")

MSFT stock news saved to aapl_stock_news.csv.


In [14]:
import os

folder_path = f'../Prepared Data/Stock News/{ticker}/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


In [15]:
from datetime import datetime

# Current date and time, formatted as YYYY-MM-DD_HHMMSS
now = datetime.now().strftime("%Y-%m-%d_%H%M%S")

In [16]:
file_path = os.path.join(folder_path, f'{ticker}_stock_news_{now}.csv')
news_df.to_csv(file_path, index=False)
print(f"File saved to {file_path}")


File saved to ../Prepared Data/Stock News/MSFT/MSFT_stock_news_2024-03-25_163333.csv
